In [1]:
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold

import tensorflow as tf
cpu_count()

4

In [2]:
# Load data frame

train_df = pd.read_csv('/kaggle/input/australian-fingerspelling/train.csv')
train_df

,path,file_id,sequence_id,participant_id,phrase
0,landmarks/101.parquet,101,10101,1,abcdefghijklmnopqrstuvwxyz
1,landmarks/101.parquet,101,10104,1,antecedent
2,landmarks/101.parquet,101,10105,1,antifascist
3,landmarks/101.parquet,101,10106,1,cooperate
4,landmarks/101.parquet,101,10107,1,describe
...,...,...,...,...,...
3097,landmarks/312.parquet,312,31292,5,nunawading
3098,landmarks/312.parquet,312,31293,5,somalia
3099,landmarks/312.parquet,312,31294,5,tokyo
3100,landmarks/312.parquet,312,31295,5,yarra


In [3]:
temp = pd.read_parquet('/kaggle/input/australian-fingerspelling/landmarks/101.parquet')
temp

,frame,sequence_id,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
1,1,10101,0.386425,0.386046,0.386145,0.383024,0.385855,0.385589,0.384911,0.364171,...,-0.009946,-0.010004,-0.007199,-0.009539,-0.006974,-0.005153,-0.009149,-0.009066,-0.005984,-0.003608
4,2,10101,0.387526,0.386954,0.387151,0.383922,0.386737,0.386461,0.385760,0.365443,...,-0.016328,-0.018092,-0.011789,-0.016041,-0.014370,-0.012743,-0.015030,-0.016318,-0.013779,-0.011740
7,3,10101,0.388560,0.387876,0.388139,0.384882,0.387642,0.387376,0.386746,0.366467,...,-0.023372,-0.021855,-0.017763,-0.023876,-0.019103,-0.014390,-0.022050,-0.023017,-0.017505,-0.013240
10,4,10101,0.389577,0.388640,0.388989,0.385550,0.388371,0.388074,0.387399,0.366969,...,-0.020876,-0.019681,-0.016725,-0.021370,-0.016427,-0.012090,-0.020847,-0.021081,-0.015429,-0.011245
13,5,10101,0.389694,0.388630,0.389059,0.385712,0.388375,0.388131,0.387623,0.367540,...,-0.019346,-0.018175,-0.015666,-0.020022,-0.015669,-0.011632,-0.019722,-0.019658,-0.014135,-0.009925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11664,55,10196,0.391923,0.391433,0.391383,0.387923,0.391161,0.390687,0.389406,0.368484,...,-0.022612,-0.022541,-0.019024,-0.025816,-0.022165,-0.018897,-0.024441,-0.027161,-0.022706,-0.019107
11667,56,10196,0.392131,0.391722,0.391633,0.388220,0.391460,0.390990,0.389710,0.368776,...,-0.022727,-0.022359,-0.018637,-0.025970,-0.022207,-0.018529,-0.024045,-0.027085,-0.022630,-0.018858
11670,57,10196,0.392516,0.392092,0.392000,0.388582,0.391836,0.391362,0.390062,0.369137,...,-0.023510,-0.023196,-0.019498,-0.026483,-0.023031,-0.019730,-0.024867,-0.027495,-0.023014,-0.019337
11673,58,10196,0.392722,0.392405,0.392246,0.388874,0.392158,0.391677,0.390332,0.369208,...,-0.019305,-0.019512,-0.016525,-0.020976,-0.017938,-0.015822,-0.021335,-0.022041,-0.018062,-0.015396


In [4]:
temp.loc[temp.index==10101]

,frame,sequence_id,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
10101,44,10183,0.386248,0.385456,0.385633,0.381855,0.385099,0.384607,0.38337,0.362572,...,-0.015255,-0.017662,-0.008397,-0.013327,-0.015946,-0.017466,-0.010537,-0.014949,-0.016717,-0.017615


In [5]:
import numpy as np
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [6]:
import json
with open('/kaggle/input/australian-fingerspelling/character_to_prediction_index.json') as json_file:
    LABEL_DICT = json.load(json_file)

In [7]:
selected_columns = temp.drop(columns=['frame']).columns

def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=selected_columns)

def encode_strings(string_list, mapping=LABEL_DICT):
    return [[mapping[char] for char in string] for string in string_list]
    
selected_columns

Index(['sequence_id', 'x_face_0', 'x_face_1', 'x_face_2', 'x_face_3',
       'x_face_4', 'x_face_5', 'x_face_6', 'x_face_7', 'x_face_8',
       ...
       'z_right_hand_11', 'z_right_hand_12', 'z_right_hand_13',
       'z_right_hand_14', 'z_right_hand_15', 'z_right_hand_16',
       'z_right_hand_17', 'z_right_hand_18', 'z_right_hand_19',
       'z_right_hand_20'],
      dtype='object', length=1630)

In [8]:
encode_strings(['asdif'])

[[32, 50, 35, 40, 37]]

In [9]:
def encode_row(row):
    coordinates = load_relevant_data_subset(f'/kaggle/input/australian-fingerspelling/{row.path}')
    coordinates = coordinates.loc[coordinates.index==row.sequence_id].values.astype('float32')
    coordinates_encoded = coordinates.tobytes()
    participant_id = int(row.participant_id)
    sequence_id = int(row.sequence_id)
#     sign = int(LABEL_DICT[row.sign])
    phrase = row.phrase.encode('utf-8')
    phrase_encoded = encode_strings([row.phrase])[0]
    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
                'file_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
                'phrase': tf.train.Feature(bytes_list=tf.train.BytesList(value=[phrase])),
                'phrase_encoded': tf.train.Feature(int64_list=tf.train.Int64List(value=phrase_encoded)),
#                 'sign':tf.train.Feature(int64_list=tf.train.Int64List(value=[sign])),
                })).SerializeToString()
    return record_bytes

def process_chunk(chunk, tfrecord_name):
    options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9)
    with tf.io.TFRecordWriter(tfrecord_name, options=options) as file_writer:
        for i, row in tqdm(chunk.iterrows()):
            record_bytes = encode_row(row)
            file_writer.write(record_bytes)
            del record_bytes
        file_writer.close()

In [10]:
row = train_df.iloc[0]

N_FILES = len(train_df)
CHUNK_SIZE = 512
N_PART = 1
FOLD = 4
part = 0

class CFG:
    seed = 42
    n_splits = 5

In [11]:
train_folds = train_df.copy()
train_folds['fold']=-1

# num_bins = 5

train_folds = train_folds.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)
# gkfold = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
# print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
# for fold_idx, (train_idx, valid_idx) in enumerate(gkfold.split(train_folds, y=train_folds['sign'].values, groups=train_folds.participant_id)):
#     train_folds.loc[valid_idx,'fold'] = fold_idx
#     print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
# kfold = KFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
# print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
# for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds)):
#     train_folds.loc[valid_idx,'fold'] = fold_idx
#     print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
kfold = GroupKFold(n_splits=CFG.n_splits) 
print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds, groups=train_folds.participant_id)):
    train_folds.loc[valid_idx,'fold'] = fold_idx
    print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
    
assert not (train_folds['fold']==-1).sum()
assert len(np.unique(train_folds['fold']))==CFG.n_splits
train_folds.head()

5fold training 3102 samples
fold0: train 2256 valid 846
fold1: train 2256 valid 846
fold2: train 2538 valid 564
fold3: train 2538 valid 564
fold4: train 2820 valid 282


,path,file_id,sequence_id,participant_id,phrase,fold
0,landmarks/301.parquet,301,30150,1,generate,3
1,landmarks/105.parquet,105,10584,2,beijing,1
2,landmarks/209.parquet,209,20937,4,culmination,2
3,landmarks/304.parquet,304,30466,2,slowly,1
4,landmarks/107.parquet,107,10701,3,abcdefghijklmnopqrstuvwxyz,0


In [12]:
%%time

import json
import os

DATASET_NAME = f'AusSLFR-{CFG.n_splits}fold'
### Create Kaggle Dataset if not exists 
# DATASET_NAME='nearestls'

!rm -rf /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('/kaggle/input/kaggleapi/kaggle.json') as f:
    kaggle_creds = json.load(f)
    
os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

!kaggle datasets init -p /tmp/{DATASET_NAME}

with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)

dataset_meta['id'] = f'jiaweiliao1/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!cp /tmp/{DATASET_NAME}/dataset-metadata.json /tmp/{DATASET_NAME}/meta.json
!ls /tmp/{DATASET_NAME}

!kaggle datasets create -p /tmp/{DATASET_NAME} --public

Data package template written to: /tmp/AusSLFR-5fold/dataset-metadata.json
{'title': 'AusSLFR-5fold', 'id': 'jiaweiliao1/AusSLFR-5fold', 'licenses': [{'name': 'CC0-1.0'}]}
dataset-metadata.json  meta.json
Starting upload for file meta.json
100%|██████████████████████████████████████████| 96.0/96.0 [00:00<00:00, 250B/s]
Upload successful: meta.json (96B)
Dataset creation error: The requested title "AusSLFR-5fold" is already in use by a dataset. Please choose another title.
CPU times: user 80.2 ms, sys: 96.9 ms, total: 177 ms
Wall time: 7.87 s


In [13]:
# Put every image in a seperate TFRecord file
# Make Pairs of Views as input to the model


def split_dataframe(df, chunk_size=10000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size:(i + 1) * chunk_size])
    return chunks

for fold in range(CFG.n_splits):
    rows = train_folds[train_folds['fold'] == fold]
    chunks = split_dataframe(rows, CHUNK_SIZE)
    
    part_size = len(chunks) // N_PART
    last = (part + 1) * part_size if part != N_PART - 1 else len(chunks)  # Changed from len(chunks) + 1
    chunks = chunks[part * part_size:last]

    # Add diagnostics here:
    # print(f"Fold: {fold}, Part: {part}")
    # print(f"Chunks: {chunks}")  # Should print list of DataFrame chunks
    if not chunks:
        print(f"Empty chunks list for part {part} in fold {fold}")
    N = [len(x) for x in chunks]
    # print(f"Lengths of chunks: {N}")

    def safe_process_chunk(x, filepath):
        try:
            process_chunk(x, filepath)
        except KeyError as e:
            print(f"KeyError: {e} in fold {fold}, chunk with length {len(x)}")

    _ = Parallel(n_jobs=cpu_count())(
        delayed(safe_process_chunk)(x, f'/tmp/{DATASET_NAME}/fold{fold}-{i}-{n}.tfrecords')
        for i, (x, n) in enumerate(zip(chunks, N))
    )


451it [03:45,  2.32it/s]
512it [04:11,  2.03it/s]
0it [00:00, ?it/s]
512it [03:08,  2.71it/s]
351it [02:16,  2.69it/s]
512it [03:14,  2.64it/s]
282it [02:09,  2.18it/s]

Upload tfrecords to usable kaggle dataset

In [14]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

20240926-095631


In [15]:
!kaggle datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip

Starting upload for file fold1-1-334.tfrecords
100%|█████████████████████████████████████████| 203k/203k [00:00<00:00, 324kB/s]
Upload successful: fold1-1-334.tfrecords (203KB)
Starting upload for file fold0-1-334.tfrecords
100%|█████████████████████████████████████████| 239k/239k [00:00<00:00, 355kB/s]
Upload successful: fold0-1-334.tfrecords (239KB)
Starting upload for file fold1-0-512.tfrecords
100%|█████████████████████████████████████████| 379k/379k [00:00<00:00, 487kB/s]
Upload successful: fold1-0-512.tfrecords (379KB)
Starting upload for file fold3-1-52.tfrecords
100%|██████████████████████████████████████| 19.5k/19.5k [00:00<00:00, 55.3kB/s]
Upload successful: fold3-1-52.tfrecords (19KB)
Starting upload for file fold3-0-512.tfrecords
  0%|                                                | 0.00/363k [00:00<?, ?B/s]

100%|█████████████████████████████████████████| 363k/363k [00:00<00:00, 429kB/s]
Upload successful: fold3-0-512.tfrecords (363KB)
Starting upload for file fold2-1-52.tfrecords
100%|██████████████████████████████████████| 19.5k/19.5k [00:00<00:00, 56.8kB/s]
Upload successful: fold2-1-52.tfrecords (20KB)
Starting upload for file fold2-0-512.tfrecords
100%|█████████████████████████████████████████| 177k/177k [00:00<00:00, 300kB/s]
Upload successful: fold2-0-512.tfrecords (177KB)
Starting upload for file fold0-0-512.tfrecords
100%|█████████████████████████████████████████| 331k/331k [00:00<00:00, 444kB/s]
Upload successful: fold0-0-512.tfrecords (331KB)
Starting upload for file fold4-0-282.tfrecords
100%|█████████████████████████████████████████| 191k/191k [00:00<00:00, 343kB/s]
Upload successful: fold4-0-282.tfrecords (191KB)
Starting upload for file meta.json
100%|██████████████████████████████████████████| 96.0/96.0 [00:00<00:00, 272B/s]
Upload successful: meta.json (96B)
Dataset versio